# GitPython Parts

In [2]:
from git import Repo
import os

In [3]:
repo = Repo(os.path.join(os.environ.get("HOME"), "commons-math"))

assert not repo.bare


In [4]:
repo.head.commit.message

'MATH-1672: Remove stat.ranking package\n\nFunctionality has been transferred to the Commons Statistics ranking\nmodule.\n'

In [11]:
repo.head.set_commit("HEAD~1").commit.message

'MATH-1670: Remove stat.inference package\n\nFunctionality has been transferred to the Commons Statistics inference\nmodule.\n'

In [ ]:
repo.head.commit.message

# Head has already set to HEAD~1, seems like we already checkouted to HEAD~1

'MATH-1670: Remove stat.inference package\n\nFunctionality has been transferred to the Commons Statistics inference\nmodule.\n'

In [15]:
repo.head.set_commit(repo.refs["origin/master"])
repo.head.commit.message


'MATH-1672: Remove stat.ranking package\n\nFunctionality has been transferred to the Commons Statistics ranking\nmodule.\n'

In [124]:
diff = repo.head.commit.diff(repo.head.commit.parents[0], create_patch=True)[0]

diff.a_path


'commons-math-legacy/pom.xml'

In [116]:


import re

def diff_header_parser(diff: str) -> list[int]:
    # print(diff)
    regex = re.compile(r"(@@) ([\-\+][0-9]+,[0-9]+ )+(@@)")

    m = re.search(regex, diff)
    
    if m == None:
        return [0, 0]

    group: list[str] = re.findall(r"[\-\+][1-9]+,[1-9]+", m.string)

    
    minus = 0
    plus = 0

    for x in group:
        d = re.search(r"([\-\+])([0-9]+),([0-9]+)", x)
        # print(d.groups())
        if d.groups()[0] == '-':
            minus += int(d.groups()[2])
        else:
            plus += int(d.groups()[2])



    return [minus, plus]
        


    
    

diff_header_parser(diff=diff.diff.__str__())



[11, 6]

In [127]:
import pandas as pd

repo_dict = dict.fromkeys(['commit', 'commit message'])
repo_dict['commit'] = []
repo_dict['commit message'] = []
repo_dict['commit changed file'] = []
repo_dict['commit minus line'] = []
repo_dict['commit plus line'] = []



for x in repo.iter_commits():
    repo_dict['commit'].append(x.hexsha)
    repo_dict['commit message'].append(x.message)
    changed_file = []
    try:
        diff = x.diff(x.parents[0], create_patch=True)
        plus = 0
        minus = 0
        for x in diff:
            if x.a_path != None:
                changed_file.append(x.a_path)
            else:
                changed_file.append("")
            num = diff_header_parser(diff=x.diff.__str__())
            plus += num[1]
            minus += num[0]
    except IndexError as e:
        pass
    finally:
        repo_dict['commit minus line'].append(minus)
        repo_dict['commit plus line'].append(plus)
        repo_dict['commit changed file'].append(changed_file)

    
    
repo_dict

{'commit': ['...',
  ...],
 'commit message': ['MATH-1672: Remove stat.ranking package\n\nFunctionality has been transferred to the Commons Statistics ranking\nmodule.\n',
  'MATH-1670: Remove stat.inference package\n\nFunctionality has been transferred to the Commons Statistics inference\nmodule.\n',
  'Merge pull request #259 from apache/dependabot/github_actions/github/codeql-action-3.28.13\n\nBump github/codeql-action from 3.28.12 to 3.28.13',
  'Bump github/codeql-action from 3.28.12 to 3.28.13\n\nBumps [github/codeql-action](https://github.com/github/codeql-action) from 3.28.12 to 3.28.13.\n- [Release notes](https://github.com/github/codeql-action/releases)\n- [Changelog](https://github.com/github/codeql-action/blob/main/CHANGELOG.md)\n- [Commits](https://github.com/github/codeql-action/compare/5f8171a638ada777af81d42b55959a643bb29017...1b549b9259bda1cb5ddde3b41741a82a2d15a841)\n\n---\nupdated-dependencies:\n- dependency-name: github/codeql-action\n  dependency-type: direct:produ

In [128]:

df = pd.DataFrame.from_dict(repo_dict)
df

,commit,commit message,commit changed file,commit minus line,commit plus line
0,e195ee2d084af99e53c9a8e47df8abbec85edc67,MATH-1672: Remove stat.ranking package\n\nFunc...,"[commons-math-legacy/pom.xml, commons-math-leg...",79,1008
1,95847b773741bf66d7a98c18f71596542921c56a,MATH-1670: Remove stat.inference package\n\nFu...,"[commons-math-legacy/pom.xml, , , , , , , , , ...",73,8458
2,6c6ffb5952ebbe921be7dbf7d1d601b8b7d67fa5,Merge pull request #259 from apache/dependabot...,"[.github/workflows/codeql-analysis.yml, .githu...",24,24
3,f7ca94625de99adb2ad7e07e696a327ba7b887b8,Bump github/codeql-action from 3.28.12 to 3.28...,"[.github/workflows/codeql-analysis.yml, .githu...",24,24
4,eb9cfebe960ff19b2b51f67dc46c21a4959080fc,Javadoc @since tag should be after @throws\n,[commons-math-legacy/src/main/java/org/apache/...,18,18
...,...,...,...,...,...
7241,9b7cfb86b26842a3a1a33b5142485cc19d8f1213,Added myself to STATUS and PROPOSAL\n\n\ngit-s...,"[PROPOSAL.html, STATUS.html]",15,24
7242,352f134f0ed563bdfd979b1b78fe8d48a18888e8,A maven project.xml was added for conv\n\n\ngi...,[project.xml],94,0
7243,e4694325bc52209c0700294437d4e260991e9f82,Starting source code - basic matrix operations...,"[build.xml, src/java/org/apache/commons/math/F...",1558,14
7244,925847780e3c03b04936f41a1739efe16702184d,added new commons math component\n\n\ngit-svn-...,"[.cvsignore, PROPOSAL.html, RELEASE-NOTES.txt,...",246,0
